# Conectando ao Banco

In [53]:
import psycopg2
import csv
import locale

connection = psycopg2.connect(
    dbname='projeto_bd',
    user='postgres',
    password='2805caique',
    host='localhost',
    port='5432'
)

cursor = connection.cursor()

def imprimir_tabela_ascii(dados, cabecalhos, incluir_enum=True, colunas_formatadas=None, max_decimais=None):
    """
    Imprime uma tabela ASCII com cabeçalhos e dados formatados.

    :param dados: Lista de tuplas, onde cada tupla representa uma linha.
    :param cabecalhos: Lista de strings com os nomes das colunas.
    :param incluir_enum: Booleano para incluir uma coluna de enumeração (default=True).
    :param colunas_formatadas: Lista de índices das colunas a serem formatadas com separador de milhar e vírgula decimal.
    """
    if not dados or not cabecalhos:
        print("Nenhum dado para mostrar.")
        return

    # Ajuste do índice das colunas formatadas, caso a enumeração esteja ativada
    colunas_formatadas = colunas_formatadas or []
    if incluir_enum:
        dados = [(i + 1,) + linha for i, linha in enumerate(dados)]
        cabecalhos = ["Nº"] + cabecalhos
        colunas_formatadas = [i + 1 for i in colunas_formatadas]

    num_colunas = len(cabecalhos)

    if (max_decimais == None):
        # Descobre o maior número de casas decimais entre os floats
        max_decimais = 0
        for linha in dados:
            for valor in linha:
                if isinstance(valor, float):
                    str_valor = str(valor)
                    if '.' in str_valor:
                        decimais = len(str_valor.split('.')[1])
                        max_decimais = max(max_decimais, decimais)

    # Cria nova matriz com valores formatados
    dados_formatados = []
    for linha in dados:
        nova_linha = []
        for i, valor in enumerate(linha):
            if isinstance(valor, float):
                valor_formatado = f"{valor:.{max_decimais}f}"
                if i in colunas_formatadas:
                    parte_inteira, parte_decimal = valor_formatado.split('.')
                    parte_inteira = f"{int(parte_inteira):,}".replace(",", ".")
                    valor_formatado = f"{parte_inteira},{parte_decimal}"
                nova_linha.append(valor_formatado)
            else:
                nova_linha.append(str(valor))
        dados_formatados.append(tuple(nova_linha))

    # Calcula a largura de cada coluna
    larguras = []
    for i in range(num_colunas):
        maior = max(len(str(linha[i])) for linha in dados_formatados)
        larguras.append(max(maior, len(cabecalhos[i])) + 2)

    # Imprime o cabeçalho
    linha_cabecalho = " | ".join(cabecalhos[i].ljust(larguras[i]) for i in range(num_colunas))
    print(linha_cabecalho)
    print("-" * len(linha_cabecalho))

    # Imprime os dados
    for linha in dados_formatados:
        linha_formatada = " | ".join(
            linha[i].rjust(larguras[i]) if i in colunas_formatadas or linha[i].replace('.', '', 1).replace(',', '', 1).isdigit()
            else linha[i].ljust(larguras[i])
            for i in range(num_colunas)
        )
        print(linha_formatada)


# PIB anual do Brasil desde 2000

In [35]:
q = """
    SELECT
        pa.ano,
        pa.qntd_habitantes * pib.valor as "PIB do ano"
    FROM pais p
    JOIN pib ON p.id_pais = pib.fk_pib_pais
    JOIN populacao_anual pa ON p.id_pais = pa.fk_populacao_pais AND pib.ano = pa.ano
    WHERE p.id_pais = 'BRA' AND pib.ano >= 2000
"""

cursor.execute(q)

imprimir_tabela_ascii(cursor.fetchall(), ['Ano', "Valor em dólares"], colunas_formatadas=[1], incluir_enum=False)

Ano    | Valor em dólares      
-------------------------------
  2000 |   2.437.186.537.953,35
  2001 |   2.471.060.904.082,38
  2002 |   2.546.513.793.722,57
  2003 |   2.575.565.054.195,91
  2004 |   2.723.916.688.389,39
  2005 |   2.811.140.110.353,63
  2006 |   2.922.517.179.348,10
  2007 |   3.099.910.252.408,75
  2008 |   3.257.825.539.619,70
  2009 |   3.253.727.075.279,87
  2010 |   3.498.674.885.848,49
  2011 |   3.637.727.027.648,88
  2012 |   3.707.613.989.001,28
  2013 |   3.819.021.353.167,28
  2014 |   3.838.267.380.872,20
  2015 |   3.702.171.527.337,32
  2016 |   3.580.891.701.109,98
  2017 |   3.628.262.212.788,34
  2018 |   3.692.978.375.782,75
  2019 |   3.738.061.309.550,99
  2020 |   3.615.574.000.080,66
  2021 |   3.787.769.313.378,39
  2022 |   3.902.035.082.023,20
  2023 |   4.015.524.694.998,83


# Top 10 maiores PIB médios desde 1990 (início dos registros)

In [37]:
q = """
    SELECT
        p.nome,
        continente.nome,
        SUM(pa.qntd_habitantes * pib.valor) / (2023 - 1990) as "PIB médio"
    FROM pais p
    JOIN pib ON p.id_pais = pib.fk_pib_pais
    JOIN populacao_anual pa ON p.id_pais = pa.fk_populacao_pais AND pib.ano = pa.ano
    JOIN continente ON p.fk_pais_continente = continente.id_continente
    WHERE pib.ano >= 1990
    GROUP BY p.id_pais, continente.nome
    ORDER BY "PIB médio" DESC
"""

cursor.execute(q)

resultados = cursor.fetchmany(10)

cabecalhos = ['Pais', 'Continente', 'PIB médio']

imprimir_tabela_ascii(resultados, cabecalhos, colunas_formatadas=[2], max_decimais=2)

Nº   | Pais                                                 | Continente      | PIB médio                
---------------------------------------------------------------------------------------------------------
   1 | United States of America                             | North America   |   18.098.948.296.599,6953
   2 | China, People's Republic of                          | Asia            |   12.908.361.852.476,1582
   3 | India, Republic of                                   | Asia            |    6.062.472.010.605,6074
   4 | Japan                                                | Asia            |    5.333.869.499.760,2861
   5 | Germany, Federal Republic of                         | Europe          |    4.552.745.642.805,0889
   6 | Russian Federation                                   | Europe          |    4.391.235.746.608,5737
   7 | France, French Republic                              | Europe          |    3.183.499.407.751,6680
   8 | Brazil, Federative Republic of         

# Top 10 países que mais emitiram CO2 desde 1990

In [45]:
q = """
    SELECT 
        p.nome AS nome_pais,
        sum((e.emissao * pa.qntd_habitantes)) AS "Total de toneladas emitidas"
    FROM 
        public.emissao_gee e
    JOIN 
        public.populacao_anual pa 
        ON e.ano = pa.ano AND e.fk_gee_pais = pa.fk_populacao_pais
    JOIN 
        public.pais p 
        ON e.fk_gee_pais = p.id_pais
    WHERE e.ano >= 1990
    GROUP BY nome_pais
    ORDER BY 
        "Total de toneladas emitidas" desc
"""

cursor.execute(q)

resultados = cursor.fetchmany(10)

cabecalhos = ['Pais', 'Toneladas de CO2 emitidas']

imprimir_tabela_ascii(resultados, cabecalhos, colunas_formatadas=[1])

Nº   | Pais                             | Toneladas de CO2 emitidas  
---------------------------------------------------------------------
   1 | China, People's Republic of      |          291.752.686.949,84
   2 | United States of America         |          218.494.746.134,55
   3 | India, Republic of               |           88.573.856.535,02
   4 | Brazil, Federative Republic of   |           83.808.199.582,75
   5 | Russian Federation               |           78.551.394.268,07
   6 | Indonesia, Republic of           |           56.337.235.705,34
   7 | Japan                            |           42.714.953.592,34
   8 | Germany, Federal Republic of     |           32.796.856.646,88
   9 | Canada                           |           28.385.309.238,00
  10 | Mexico, United Mexican States    |           24.541.047.592,18


# Acumulado de PIB e Toneladas de CO2 emitidas desde 1995 e relação grama de CO2 emitido por dolar "produzido" no período

In [54]:
q = """
SELECT 
	c.nome as "Continente",
	sum(emissao_gee."emissao total") as "Acumulado de Toneladas de CO2 emitidas",
	sum(pib_acumulado."PIB acumulado") as "PIB acumulado",
	(sum(emissao_gee."emissao total")*10^6)/sum(pib_acumulado."PIB acumulado") as "Grama emitido por dolar gerado"
FROM pais p
JOIN pib ON p.id_pais = pib.fk_pib_pais
JOIN ( -- subconsulta para encontrar as emissoes acumuladas desde 1995
	SELECT 
		pa.fk_populacao_pais,
		pa.ano,
		(pa.qntd_habitantes * gee_perc.emissao) as "emissao total"
	FROM populacao_anual pa
	JOIN emissao_gee gee_perc ON gee_perc.fk_gee_pais = pa.fk_populacao_pais AND gee_perc.ano = pa.ano
	WHERE pa.ano >= 1995
) emissao_gee ON emissao_gee.fk_populacao_pais = p.id_pais
JOIN continente c ON c.id_continente = p.fk_pais_continente
JOIN ( -- subconsulta para encontrar a soma dos pibs anuais desde 1995
	SELECT 
		p.id_pais,
		sum(pib.valor * pa.qntd_habitantes) as "PIB acumulado"
	FROM continente c
	JOIN pais p ON p.fk_pais_continente = c.id_continente
	JOIN pib ON pib.fk_pib_pais = p.id_pais
	JOIN populacao_anual pa ON pa.fk_populacao_pais = p.id_pais
	WHERE pa.ano >= 1995
	GROUP BY p.id_pais
) pib_acumulado ON pib_acumulado.id_pais = p.id_pais
GROUP BY c.id_continente
ORDER BY sum(emissao_gee."emissao total") DESC
"""

cursor.execute(q)

r = cursor.fetchall()

cabecalhos = [
    'Continente',
    "Acumulado de Toneladas de CO2 emitidas",
    'PIB acumulado',
    "Grama emitido por dolar gerado"
]

imprimir_tabela_ascii(r, cabecalhos, colunas_formatadas=[1,2,3], max_decimais=4)

Nº   | Continente      | Acumulado de Toneladas de CO2 emitidas   | PIB acumulado                     | Grama emitido por dolar gerado  
----------------------------------------------------------------------------------------------------------------------------------------
   1 | Asia            |                  20.536.206.589.889,4375 |   35.889.426.409.452.249.088,0000 |                           0,5722
   2 | North America   |                   8.259.219.388.661,3936 |   21.752.752.535.671.267.328,0000 |                           0,3797
   3 | Europe          |                   8.199.257.349.436,7637 |   29.701.466.684.452.507.648,0000 |                           0,2761
   4 | Africa          |                   4.142.317.560.090,8438 |    5.046.125.397.796.729.856,0000 |                           0,8209
   5 | South America   |                   3.808.707.855.549,2207 |    5.556.743.992.269.351.936,0000 |                           0,6854
   6 | Oceania         |                 